# Discovery of Frequent Itemsets and Association Rules

1. Frequent itemsets with support s
2. Association Rules with confidence c


A priori:
C1
    - read transactions and get counts
L1
    - filter candidates with support s

CK
    - generate candidates CK out of L(k-1), with monotonicity support s,
    - read transactions and get counts
LK
- filter candidates LK with support s



Keep in mind
- if the files are very large and can't be fit into memory, then these files should be split. With a dataloader

In [11]:
def load_transactions(filepath):
    transactions = []
    
    with open(filepath, 'r') as f:
        for line in f:
            # Convert each line to a set of integers
            transaction = set(map(int, line.strip().split()))
            transactions.append(transaction)
    
    return transactions

transactions = load_transactions('data/T10I4D100K.dat')
print(f"Number of transactions: {len(transactions)}")
print(f"First few transactions: {transactions[:3]}")

all_items = set().union(*transactions)
print(f"Total unique items: {len(all_items)}")

Number of transactions: 100000
First few transactions: [{448, 834, 164, 775, 328, 687, 240, 368, 274, 561, 52, 630, 825, 25, 538, 730}, {704, 834, 581, 39, 205, 814, 401, 120, 825, 124}, {674, 35, 712, 854, 759, 950, 249, 733}]
Total unique items: 870


# A Priori algorithm

the idea is to incrementally build larger itemsets by combining frequently occuring smaller subsets.


### Frequent singletons:

    1. find singletons
    2. c1 <- count singletons
    3. l1 <- filter frequent c1

### Frequent 2-itemsets

    1. candidates <- combine singletons to 2-itemsets
    2. c2 <- count candidates
    3. l2 <- filter c2

### Frequent k-itemsets
    1. candidates <- combine singletons with k-1 item sets
    2. ensure frequent subsets - they occur in all frequently filtered candidates L (consisting of each lk)
    3. ck <- count candidates
    4. lk <- filter ck



    




In [69]:
from tqdm import tqdm

def A_priori(transactions, s, max_k=3):
    # get counts for singletons (k=1)
    c1 = {frozenset([item]): 0 for item in all_items}
    for transaction in transactions:
        for item in transaction:
            c1[frozenset([item])] += 1
    
    # filter singletons with support s  
    l1 = {itemset: count for itemset, count in c1.items() if count >= s}
    
    # initialize result with L1
    L = [l1]
    C = [c1]
    
    print(f"C1: {len(c1)} - {c1}")
    print(f"L1: {len(l1)} - {l1}")
    
    # iterate for k=2 to max_k
    for k in range(2, max_k + 1):
        # generate candidates from previous frequent itemsets
        ck = {}
        prev_l = L[k-2]
        
        # generate candidates by combining previous frequent itemsets
        for itemset1 in prev_l:
            for singleton in l1:
                union = itemset1.union(singleton)
                if len(union) == k:
                    
                    # subset frequency check
                    all_subsets_frequent = True
                    for item in union:
                        subset = frozenset(union - {item})
                        if subset not in prev_l:
                            all_subsets_frequent = False
                            break
                    
                    if all_subsets_frequent:
                        ck[union] = 0
        
        print(f"C{k} candidates: {len(ck)} - {ck}")
 
        # count occurrences of candidates
        for transaction in tqdm(transactions):
            transaction_set = frozenset(transaction)
            for candidate in ck:
                if candidate.issubset(transaction_set):
                    ck[candidate] += 1
        
        print(f"C{k} counts: {len(ck)} - {ck}")
        C.append(ck)
        
        # filter candidates with minimum support
        lk = {itemset: count for itemset, count in ck.items() if count >= s}
        
        print(f"L{k}: {len(lk)} - {lk}")
        
        # if no frequent itemsets found, break
        if not lk:
            break
        L.append(lk)
            
    
    return L, C



In [70]:
s = 0.05*len(transactions)
print(f"s: {s}")
L, C = A_priori(transactions, s, 3)


s: 0.4
C1: 870 - {frozenset({0}): 0, frozenset({1}): 5, frozenset({2}): 6, frozenset({3}): 6, frozenset({4}): 2, frozenset({5}): 4, frozenset({6}): 1, frozenset({7}): 0, frozenset({8}): 0, frozenset({10}): 0, frozenset({11}): 0, frozenset({12}): 0, frozenset({13}): 0, frozenset({14}): 0, frozenset({15}): 0, frozenset({16}): 0, frozenset({17}): 0, frozenset({18}): 0, frozenset({19}): 0, frozenset({20}): 0, frozenset({21}): 0, frozenset({22}): 0, frozenset({23}): 0, frozenset({24}): 0, frozenset({25}): 0, frozenset({26}): 0, frozenset({27}): 0, frozenset({28}): 0, frozenset({29}): 0, frozenset({31}): 0, frozenset({32}): 0, frozenset({33}): 0, frozenset({34}): 0, frozenset({35}): 0, frozenset({36}): 0, frozenset({37}): 0, frozenset({38}): 0, frozenset({39}): 0, frozenset({40}): 0, frozenset({41}): 0, frozenset({42}): 0, frozenset({43}): 0, frozenset({44}): 0, frozenset({45}): 0, frozenset({46}): 0, frozenset({47}): 0, frozenset({48}): 0, frozenset({49}): 0, frozenset({51}): 0, frozenset({

100%|██████████| 8/8 [00:00<00:00, 77672.30it/s]


C2 counts: 15 - {frozenset({1, 2}): 3, frozenset({1, 3}): 4, frozenset({1, 4}): 2, frozenset({1, 5}): 2, frozenset({1, 6}): 1, frozenset({2, 3}): 5, frozenset({2, 4}): 0, frozenset({2, 5}): 3, frozenset({2, 6}): 1, frozenset({3, 4}): 1, frozenset({3, 5}): 2, frozenset({3, 6}): 1, frozenset({4, 5}): 1, frozenset({4, 6}): 0, frozenset({5, 6}): 0}
L2: 12 - {frozenset({1, 2}): 3, frozenset({1, 3}): 4, frozenset({1, 4}): 2, frozenset({1, 5}): 2, frozenset({1, 6}): 1, frozenset({2, 3}): 5, frozenset({2, 5}): 3, frozenset({2, 6}): 1, frozenset({3, 4}): 1, frozenset({3, 5}): 2, frozenset({3, 6}): 1, frozenset({4, 5}): 1}
C3 candidates: 10 - {frozenset({1, 2, 3}): 0, frozenset({1, 2, 5}): 0, frozenset({1, 2, 6}): 0, frozenset({1, 3, 4}): 0, frozenset({1, 3, 5}): 0, frozenset({1, 3, 6}): 0, frozenset({1, 4, 5}): 0, frozenset({2, 3, 5}): 0, frozenset({2, 3, 6}): 0, frozenset({3, 4, 5}): 0}


100%|██████████| 8/8 [00:00<00:00, 74235.47it/s]

C3 counts: 10 - {frozenset({1, 2, 3}): 3, frozenset({1, 2, 5}): 1, frozenset({1, 2, 6}): 1, frozenset({1, 3, 4}): 1, frozenset({1, 3, 5}): 1, frozenset({1, 3, 6}): 1, frozenset({1, 4, 5}): 1, frozenset({2, 3, 5}): 2, frozenset({2, 3, 6}): 1, frozenset({3, 4, 5}): 0}
L3: 9 - {frozenset({1, 2, 3}): 3, frozenset({1, 2, 5}): 1, frozenset({1, 2, 6}): 1, frozenset({1, 3, 4}): 1, frozenset({1, 3, 5}): 1, frozenset({1, 3, 6}): 1, frozenset({1, 4, 5}): 1, frozenset({2, 3, 5}): 2, frozenset({2, 3, 6}): 1}


In [71]:
L

[{frozenset({1}): 5,
  frozenset({2}): 6,
  frozenset({3}): 6,
  frozenset({4}): 2,
  frozenset({5}): 4,
  frozenset({6}): 1},
 {frozenset({1, 2}): 3,
  frozenset({1, 3}): 4,
  frozenset({1, 4}): 2,
  frozenset({1, 5}): 2,
  frozenset({1, 6}): 1,
  frozenset({2, 3}): 5,
  frozenset({2, 5}): 3,
  frozenset({2, 6}): 1,
  frozenset({3, 4}): 1,
  frozenset({3, 5}): 2,
  frozenset({3, 6}): 1,
  frozenset({4, 5}): 1},
 {frozenset({1, 2, 3}): 3,
  frozenset({1, 2, 5}): 1,
  frozenset({1, 2, 6}): 1,
  frozenset({1, 3, 4}): 1,
  frozenset({1, 3, 5}): 1,
  frozenset({1, 3, 6}): 1,
  frozenset({1, 4, 5}): 1,
  frozenset({2, 3, 5}): 2,
  frozenset({2, 3, 6}): 1}]

In [112]:
c2 = sorted([(v,c) for v,c in C[1].items()], key=lambda x: x[1], reverse=True)
print(c2)


[(frozenset({217, 346}), 1336), (frozenset({368, 829}), 1194), (frozenset({829, 789}), 1194), (frozenset({368, 682}), 1193), (frozenset({825, 39}), 1187), (frozenset({529, 598}), 943), (frozenset({368, 692}), 928), (frozenset({217, 283}), 926), (frozenset({346, 283}), 910), (frozenset({368, 494}), 860), (frozenset({684, 438}), 825), (frozenset({75, 684}), 797), (frozenset({75, 775}), 787), (frozenset({75, 438}), 772), (frozenset({722, 12}), 740), (frozenset({145, 39}), 734), (frozenset({32, 947}), 720), (frozenset({862, 494}), 720), (frozenset({217, 947}), 716), (frozenset({682, 494}), 714), (frozenset({368, 937}), 701), (frozenset({541, 494}), 694), (frozenset({888, 346}), 689), (frozenset({120, 895}), 685), (frozenset({966, 438}), 676), (frozenset({766, 71}), 673), (frozenset({883, 494}), 671), (frozenset({120, 862}), 665), (frozenset({510, 487}), 646), (frozenset({937, 509}), 643), (frozenset({368, 529}), 640), (frozenset({862, 895}), 635), (frozenset({145, 419}), 631), (frozenset({

# Association Rules

1. frequent itemset I
2. rule generation
    - for subset A in I:
    
        1. conf(A -> A/I) = supp(I) / supp(A)              given A how likely to also observe A/I

        2. if conf > c:

            We've found a rule!

In [76]:
transactions = [
    {1, 2, 3},    # B1 {m, c, b}
    {1, 4, 5},    # B2 {m, p, j}
    {1, 2, 3, 6}, # B3 {m, c, b, n}
    {2, 5},       # B4 {c, j}
    {1, 4, 3},    # B5 {m, p, b}
    {1, 2, 3, 5}, # B6 {m, c, b, j}
    {2, 3, 5},    # B7 {c, b, j}
    {3, 2}        # B8 {b, c}
]


L, C = A_priori(transactions, s=3, max_k=3)

C1: 870 - {frozenset({0}): 0, frozenset({1}): 5, frozenset({2}): 6, frozenset({3}): 6, frozenset({4}): 2, frozenset({5}): 4, frozenset({6}): 1, frozenset({7}): 0, frozenset({8}): 0, frozenset({10}): 0, frozenset({11}): 0, frozenset({12}): 0, frozenset({13}): 0, frozenset({14}): 0, frozenset({15}): 0, frozenset({16}): 0, frozenset({17}): 0, frozenset({18}): 0, frozenset({19}): 0, frozenset({20}): 0, frozenset({21}): 0, frozenset({22}): 0, frozenset({23}): 0, frozenset({24}): 0, frozenset({25}): 0, frozenset({26}): 0, frozenset({27}): 0, frozenset({28}): 0, frozenset({29}): 0, frozenset({31}): 0, frozenset({32}): 0, frozenset({33}): 0, frozenset({34}): 0, frozenset({35}): 0, frozenset({36}): 0, frozenset({37}): 0, frozenset({38}): 0, frozenset({39}): 0, frozenset({40}): 0, frozenset({41}): 0, frozenset({42}): 0, frozenset({43}): 0, frozenset({44}): 0, frozenset({45}): 0, frozenset({46}): 0, frozenset({47}): 0, frozenset({48}): 0, frozenset({49}): 0, frozenset({51}): 0, frozenset({52}): 0

100%|██████████| 8/8 [00:00<00:00, 79324.90it/s]


C2 counts: 6 - {frozenset({1, 2}): 3, frozenset({1, 3}): 4, frozenset({1, 5}): 2, frozenset({2, 3}): 5, frozenset({2, 5}): 3, frozenset({3, 5}): 2}
L2: 4 - {frozenset({1, 2}): 3, frozenset({1, 3}): 4, frozenset({2, 3}): 5, frozenset({2, 5}): 3}
C3 candidates: 1 - {frozenset({1, 2, 3}): 0}


100%|██████████| 8/8 [00:00<00:00, 142784.82it/s]

C3 counts: 1 - {frozenset({1, 2, 3}): 3}
L3: 1 - {frozenset({1, 2, 3}): 3}


In [111]:

import itertools
from typing import List, Dict, FrozenSet

def find_association_rules(L:List[Dict[FrozenSet[int], int]], c=0.5):
    '''
    L: list of frequent itemsets (L1, L2, ... Lk), where Lk is a dict of itemsets and their counts
    '''
    
    # get all frequent itemsets
    frequent_itemsets = {}
    for level in L:
        frequent_itemsets.update(level)
    
    # filter out singletons
    frequent_itemsets = list(k for k in frequent_itemsets.keys() if len(k) > 1)
    
    # support counts
    counts = {}
    for level in L:
        counts.update(level)
    
    # generate rules
    rules = {}
    for itemset in frequent_itemsets:
        n = len(itemset)
        for i in range(1, n):
            for subset in map(frozenset, itertools.combinations(itemset, i)):
                consequent = itemset - subset
                if len(consequent) > 0:  # Ensure non-empty consequent
                    conf = counts[itemset] / counts[subset]
                    if conf >= c:
                        rules[(subset, consequent)] = conf
    
    return rules


In [108]:
find_association_rules(L, c=0.75)

[frozenset({1, 2}), frozenset({1, 3}), frozenset({2, 3}), frozenset({2, 5}), frozenset({1, 2, 3})]
{frozenset({1}): 5, frozenset({2}): 6, frozenset({3}): 6, frozenset({5}): 4, frozenset({1, 2}): 3, frozenset({1, 3}): 4, frozenset({2, 3}): 5, frozenset({2, 5}): 3, frozenset({1, 2, 3}): 3}


{(frozenset({1}), frozenset({3})): 0.8,
 (frozenset({2}), frozenset({3})): 0.8333333333333334,
 (frozenset({3}), frozenset({2})): 0.8333333333333334,
 (frozenset({5}), frozenset({2})): 0.75,
 (frozenset({1, 2}), frozenset({3})): 1.0,
 (frozenset({1, 3}), frozenset({2})): 0.75}

# Putting it all together

1. Find frequent itemsets
2. Find association rules

In [113]:
transactions = load_transactions('data/T10I4D100K.dat')

s = 0.01*len(transactions)
print(f"s: {s}")
L, C = A_priori(transactions, s, 3)

find_association_rules(L, c=0.75)



s: 1000.0
C1: 870 - {frozenset({0}): 594, frozenset({1}): 1535, frozenset({2}): 673, frozenset({3}): 531, frozenset({4}): 1394, frozenset({5}): 1094, frozenset({6}): 2149, frozenset({7}): 997, frozenset({8}): 3090, frozenset({10}): 1351, frozenset({11}): 525, frozenset({12}): 3415, frozenset({13}): 35, frozenset({14}): 197, frozenset({15}): 458, frozenset({16}): 150, frozenset({17}): 1683, frozenset({18}): 813, frozenset({19}): 121, frozenset({20}): 40, frozenset({21}): 2666, frozenset({22}): 397, frozenset({23}): 128, frozenset({24}): 191, frozenset({25}): 1395, frozenset({26}): 527, frozenset({27}): 2165, frozenset({28}): 1454, frozenset({29}): 171, frozenset({31}): 1666, frozenset({32}): 4248, frozenset({33}): 1460, frozenset({34}): 56, frozenset({35}): 1984, frozenset({36}): 528, frozenset({37}): 1249, frozenset({38}): 2402, frozenset({39}): 4258, frozenset({40}): 457, frozenset({41}): 1353, frozenset({42}): 119, frozenset({43}): 1721, frozenset({44}): 903, frozenset({45}): 1728, f

100%|██████████| 100000/100000 [03:12<00:00, 519.68it/s]


C2 counts: 70125 - {frozenset({1, 4}): 15, frozenset({1, 5}): 9, frozenset({1, 6}): 23, frozenset({8, 1}): 177, frozenset({1, 10}): 12, frozenset({1, 12}): 37, frozenset({1, 17}): 17, frozenset({1, 21}): 33, frozenset({1, 25}): 9, frozenset({1, 27}): 18, frozenset({1, 28}): 10, frozenset({1, 31}): 10, frozenset({32, 1}): 125, frozenset({1, 33}): 14, frozenset({1, 35}): 21, frozenset({1, 37}): 12, frozenset({1, 38}): 22, frozenset({1, 39}): 31, frozenset({1, 41}): 17, frozenset({1, 43}): 19, frozenset({1, 45}): 19, frozenset({48, 1}): 305, frozenset({1, 51}): 15, frozenset({1, 52}): 20, frozenset({1, 54}): 365, frozenset({1, 55}): 13, frozenset({1, 57}): 38, frozenset({1, 58}): 13, frozenset({1, 68}): 21, frozenset({1, 69}): 29, frozenset({1, 70}): 24, frozenset({1, 71}): 38, frozenset({72, 1}): 40, frozenset({73, 1}): 20, frozenset({1, 75}): 28, frozenset({1, 78}): 27, frozenset({1, 85}): 18, frozenset({1, 90}): 22, frozenset({1, 93}): 21, frozenset({1, 94}): 19, frozenset({1, 97}): 10

100%|██████████| 100000/100000 [00:00<00:00, 1954001.83it/s]

C3 counts: 1 - {frozenset({704, 825, 39}): 1035}
L3: 1 - {frozenset({704, 825, 39}): 1035}


{(frozenset({704, 825}), frozenset({39})): 0.9392014519056261,
 (frozenset({39, 704}), frozenset({825})): 0.9349593495934959,
 (frozenset({39, 825}), frozenset({704})): 0.8719460825610783}

In [114]:
find_association_rules(L, c=0.5)

{(frozenset({704}), frozenset({39})): 0.617056856187291,
 (frozenset({227}), frozenset({390})): 0.577007700770077,
 (frozenset({704}), frozenset({825})): 0.6142697881828316,
 (frozenset({704}), frozenset({39, 825})): 0.5769230769230769,
 (frozenset({704, 825}), frozenset({39})): 0.9392014519056261,
 (frozenset({39, 704}), frozenset({825})): 0.9349593495934959,
 (frozenset({39, 825}), frozenset({704})): 0.8719460825610783}

# Conclusion
The association rules suggests that if 704 and 825 is bought, then 39 is also bought, with a high condifence of almost 94%. There seem to be a strong correlation between all of these three products, while it seems like 39 and 825 is more often bought without 704 than the others. 

if 704 is bought, then you can be quite confident of the customer also buying 30 or 825 ~60%
if 704 is bought with either 39 or 825, then with a confidence of more than 93% we can say that they'll also buy all three. 


It would be clever to arrange all of these products close to each other in the store.

226 and 390 also seems to have some correlation, yet lower at conf~58%